In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 41.6 MB/s eta 0:00:00


In [2]:
import mysql.connector
import re
import pandas as pd

In [ ]:
# Connect to MySQL database
def connect_to_db():
    return mysql.connector.connect(
        host="your_host",
        user="your_username",
        password="your_password",
        database="your_database"
    )

# Execute SQL query and return result
def execute_sql_query(query):
    db = connect_to_db()
    cursor = db.cursor()
    cursor.execute(query)
    result = cursor.fetchone()[0]
    db.close()
    return result

In [ ]:
def extract_queries_from_file(file_path):
    queries = []
    pattern = re.compile(r"Value: (\d+), SQL Query: (.+)")

    with open(file_path, 'r') as file:
        for line in file:
            match = pattern.search(line)
            if match:
                value = match.group(1)
                sql_query = match.group(2)
                queries.append((value, sql_query))

    return queries

In [10]:
def parse_query(query_string):
    # Extract the value and query from the string
    value_part, query_part = query_string.split(", SQL Query: ")
    value = int(value_part.split(": ")[1])
    query = query_part.strip()
    return value, query

In [11]:
def validate_queries(queries, threshold):
    results = []

    for query_string in queries:
        value, query = parse_query(query_string)
        # query_result = execute_sql_query(query)
        query_result = 0  # This is a placeholder, replace with actual query execution
        difference = abs(value - query_result)
        pass_fail = "Pass" if difference <= value * threshold else "Fail"
        results.append({
            "Expected Value": value,
            "Query Result": query_result,
            "Difference": difference,
            "Result": pass_fail
        })

    return pd.DataFrame(results)

In [15]:
reach_queries = ["Value: 2370069038, SQL Query: select sum(editorial_reach) as editorial_reach from ads_people_metrics.yt_reach_artifacts.2024 where country_code = 'US' and device = 'CTV' and model like 'quito_%' AND start_date = 20240401 and age_range NOT IN ('0-17')", "Value: 2427415712, SQL Query: SELECT SUM(editorial_reach) as editorial_reach FROM ads_people_metrics.yt_reach_artifacts.2024 WHERE country_code = 'US' AND device = 'CTV' AND model LIKE 'quito_%' AND start_date = 20240501 AND age_range NOT IN ('0-17')"];

In [13]:
watch_time_queries = ["Value: 2363666772, SQL Query: SELECT MAX(watch_time_min) AS watch_time_min FROM yt_mp_relaylog_data.FirstPartyWatchViews.all WHERE start_date IN ('2024-04') AND platform = 'DESKTOP_MOBILE_TABLET_TV' AND country = 'US' AND partner = 'NIELSEN' AND min_sec = 0 AND timezone = 'America/New_York';", "Value: 2421175412, SQL Query: SELECT MAX(watch_time_min) AS watch_time_min FROM yt_mp_relaylog_data.FirstPartyWatchViews.all WHERE start_date IN ('2024-05') AND platform = 'DESKTOP_MOBILE_TABLET_TV' AND country = 'US' AND partner = 'NIELSEN' AND min_sec = 0 AND timezone = 'America/New_York';"];

In [16]:
# reach_file_path = 'reach_sql_queries.txt'
# watch_time_file_path = 'watch_time_sql_queries.txt'

# reach_queries = extract_queries_from_file(reach_file_path)
# watch_time_queries = extract_queries_from_file(watch_time_file_path)

watch_time_results = validate_queries(watch_time_queries, 0.1)
reach_results = validate_queries(reach_queries, 0.1)
print("REACH RESULTS:")
print(reach_results)
print("WATCH TIME RESULTS:")
print(watch_time_results)

REACH RESULTS:
   Expected Value  Query Result  Difference Result
0      2370069038             0  2370069038   Fail
1      2427415712             0  2427415712   Fail
WATCH TIME RESULTS:
   Expected Value  Query Result  Difference Result
0      2363666772             0  2363666772   Fail
1      2421175412             0  2421175412   Fail
